In [1]:
import pandas as pd
import math

## Step 1: Gather and Prepare Data

### Input Data

#### Heavy Machiney Data

In [29]:
# Load the dataset
file_path = 'C:\\Users\\ASUS\\Desktop\\OverviewMachines.xlsx'
df = pd.read_excel(file_path)

# Column extraction
included_columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 12']  # Adjust according to your dataset

# Change column names
column_titles = {
    'Unnamed: 1': 'Machine Name',
    'Unnamed: 2': 'Machine Type',
    'Unnamed: 3': 'Battery Capacity (kWh)',
    'Unnamed: 4': 'Charging Speed (kW)',
    'Unnamed: 7': 'Total Charging Demand for Normal Working (kWh)',
    'Unnamed: 12': 'Total Charging Demand for Heavy Working (kWh)'
    
}

# Input machine number
machine_number = int(input("\nEnter the machine number (starting from 1): ")) 

# Convert machine number to actual dataset row index
actual_row_number = machine_number + 1  

# Validate machine number
if machine_number < 1 or actual_row_number > len(df):
    print(f"\n❌ Invalid machine number. Please enter a number between 1 and {len(df) - 1}.")
elif not all(col in df.columns for col in included_columns):
    print(f"\n❌ Invalid columns found in {included_columns}. Please check the column names in the dataset.")
else:
    # Extract values for the specified columns and row
    machine_data = df.loc[actual_row_number - 1, included_columns].to_dict()

    # Rename keys using custom column names
    machine_data = {column_titles[k]: v for k, v in machine_data.items()}

    # Store values in separate variables
    Machine_name = machine_data['Machine Name']
    Machine_type = machine_data['Machine Type']
    battery_capacity = machine_data['Battery Capacity (kWh)']
    
    # Since nominal capacities are limited due to battery longevity, the usable battery capacity is lower.
    battery_capacity_real = battery_capacity * 0.9                       # Assumed usable capacity is 90% of the nominal one.   
    
    charging_speed_vehicle = machine_data['Charging Speed (kW)']
    

    print(f"\n✅ Data for machine:")
    print(f"Machine name: {Machine_name}")
    print(f"Machine type: {Machine_type}")
    print(f"Nominal Battery Capacity: {battery_capacity} kWh, Usable Battery Capacity {battery_capacity_real}")
    print(f"Charging Speed: {charging_speed_vehicle} kW")
    


Enter the machine number (starting from 1): 1

✅ Data for machine:
Machine name: Cat 320 Z-Line
Machine type: 25-tonne Excavator
Nominal Battery Capacity: 300 kWh, Usable Battery Capacity 270.0
Charging Speed: 40 kW


#### Input Remaining Battery

In [30]:
battery_percentage = int(input('\nEnter the battery percentage on screen (from 0 to 100): '))
if battery_percentage < 0 or battery_percentage > 100:
    print(f"\n❌ State of the charge number. Please enter a number between 0 and 100.")
else:
    battery_percentage = battery_percentage / 100
    SOC = battery_capacity_real * battery_percentage
    Required_energy = battery_capacity_real - SOC
    print (f'The machine requires {Required_energy} kwh energy')


Enter the battery percentage on screen (from 0 to 100): 0
The machine requires 270.0 kwh energy


#### Stations Data

In [31]:
dataset = pd.read_excel('C:\\Users\\ASUS\\Downloads\\archive\\Final_dataset(Cleaned_extra_stations).xlsx')
dataset
Charging_speed_station = dataset['capacity']             # Charging speed (Kw)
Availability = dataset['available']             # Availability
latitude = dataset['latitude']       # Latitudes of stations
longitude = dataset['longitude']     # Longitudes of stations

## Step 2: Define the rules

#### Distance Check

In [32]:
from geopy.distance import geodesic

# Reference coordinator (construction field site)
reference_point = (28.748103, 77.112122)   

# Calculate the distance of the construction site to stations by Vincenty formula based (by geopy) on their geographical coordinators
dataset['Distance'] = dataset.apply(lambda row: round(geodesic(reference_point, 
                                                    (row['latitude'], row['longitude'])).kilometers, 2), axis=1)

# Filter stations by distance condition (within 5 km)                                        
results_distance = []
for _, row in dataset[dataset['Distance'] <= 5].iterrows():
    results_distance.append([row['name'], row['address'], 
                             row['latitude'], row['longitude'], row['cost_per_unit'], 
                             row['Distance'], row.get('available', None)])

# Convert to dataframe 
filtered_distance_df = pd.DataFrame(results_distance,                                      
                                    columns=['Name', 'Address', 'Latitude', 'Longitude', 
                                             'Cost_per_Unit', 'Distance', 'Available'])   

# Reset the index to start from 1
filtered_distance_df.index = range(1, len(filtered_distance_df) + 1)
# Display dataset
filtered_distance_df.head()

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
1,REVOS,sector 14 rohini,28.717569,77.125984,10.0,3.64,1
2,REVOS,Pocket 3 Sector 23 Rohini Delhi,28.720940,77.080181,10.0,4.34,1
3,REVOS,Pocket 3 Sector 23 Rohini Delhi,28.721847,77.078821,10.0,4.36,1
4,REVOS,"plot no-1,khasra no-768/1, siraspur",28.760563,77.126411,10.0,1.96,1
5,REVOS,A 59 Gali No 6 Suraj Park Delhi 110042,28.743411,77.135582,10.0,2.35,1


#### Availability Check

In [33]:
# Filter stations for their availability
results_available = []
for _, row in dataset[dataset['available'] > 0].iterrows():                     
    results_available.append([row['name'], row['address'], row['latitude'], row['longitude'], row['cost_per_unit'], row.get('Distance', None), row['available']])

# Convert to dataframe
filtered_available_df = pd.DataFrame(results_available,                        
                                     columns=['Name', 'Address', 'Latitude', 'Longitude', 'Cost_per_Unit', 'Distance', 'Available'])

# Reset the index to start from 1
filtered_available_df.index = range(1, len(filtered_available_df) + 1)
# Display dataset
filtered_available_df.head()

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
1,EESL,"NDMC Parking, Outside Jain Bhawan, Shaheed B...",28.633614,77.207691,10.0,15.76,1
2,EESL,"PSOI Club, Chanakyapuri, New Delhi 110021",28.586638,77.193352,9.5,19.58,1
3,EESL,Outside Chelmsford Club/ Opposite CSIR Buildin...,28.628177,77.212590,9.5,16.52,2
4,EESL,"Opposite ICICI Bank, Janpath, New Delhi",28.628528,77.219460,9.5,16.90,1
5,EESL,Talkatora Stadium Presidents Estate New Delhi ...,28.625753,77.194530,9.5,15.77,1


#### Combining Two Conditions

In [34]:
# Combine both two conditions
results_combined = []
for _, row in dataset[(dataset['Distance'] <= 10) & (dataset['available'] > 0)].iterrows():
    results_combined.append([row['name'], row['address'], row['latitude'], row['longitude'], row['cost_per_unit'], row['Distance'], row['available']])

# Convert to dataframe
filtered_combined_df = pd.DataFrame(results_combined,                                       
                                    columns=['Name', 'Address', 'Latitude', 'Longitude', 'Cost_per_Unit', 'Distance', 'Available'])
# Reset the index to start from 1
filtered_combined_df.index = range(1, len(filtered_combined_df) + 1)


# Display dataset
filtered_combined_df.head()

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
1,REVOS,142 Deepali Enclave Pitampura Delhi 110034,28.694492,77.118912,10.0,5.98,1
2,REVOS,sector 14 rohini,28.717569,77.125984,10.0,3.64,1
3,REVOS,"473, Britannia Rd, Tri Nagar, Delhi, 110034",28.680616,77.155446,10.0,8.59,1
4,REVOS,"Nangloi, Delhi, 110041",28.681211,77.075725,10.0,8.22,1
5,REVOS,"473, Britannia Rd, Tri Nagar, Delhi, 110034zypp",28.680721,77.155315,5.0,8.58,1


In [35]:
# Display all stations dataset within range and availability
filtered_combined_df

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
1,REVOS,142 Deepali Enclave Pitampura Delhi 110034,28.694492,77.118912,10.00,5.98,1
2,REVOS,sector 14 rohini,28.717569,77.125984,10.00,3.64,1
3,REVOS,"473, Britannia Rd, Tri Nagar, Delhi, 110034",28.680616,77.155446,10.00,8.59,1
4,REVOS,"Nangloi, Delhi, 110041",28.681211,77.075725,10.00,8.22,1
5,REVOS,"473, Britannia Rd, Tri Nagar, Delhi, 110034zypp",28.680721,77.155315,5.00,8.58,1
...,...,...,...,...,...,...,...
252,E-Fill Electric,"A-38, DSIIDC, Bawana Industrial Area, Sec-5",28.792700,77.072724,15.00,6.26,1
253,E-Fill Electric,"Akash Ganga Apartment, Pocket-4, Rohini Sector-24",28.726456,77.083414,15.00,3.69,1
254,HP Pitampura DC Charger-1,"HP PETROL PUMP - AUTO CARE CENTRE, Pitam Pura,...",28.696200,77.155000,18.00,7.12,2
255,Singh Service Station,GT Karnal Road DELHI-110075,28.691400,77.189300,2.05,9.82,2


#### Number of Acceptable Stations

In [36]:
# Display the total number of accepted stations that have both availability and distance check conditions
filtered_combined_df.shape
print(f"\nThe number of accepted stations: {len(filtered_combined_df)} ")


The number of accepted stations: 256 


#### Display the Stations on Map

In [37]:
import plotly.graph_objects as go

# Reference point coordinates
reference_point = {"latitude": reference_point[0], "longitude": reference_point[1], "name": "Reference Point"}

# Create the map figure
fig = go.Figure()

# Add the reference point as a larger marker
fig.add_trace(go.Scattermapbox(
    lat=[reference_point['latitude']],
    lon=[reference_point['longitude']],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=15,
        color='red'  
    ),
    name=reference_point['name'],             # legend box
    hoverinfo='text',
    text=f"<b>{reference_point['name']}</b>"  # Tooltip text
))

# Add the chosen stations as smaller markers
fig.add_trace(go.Scattermapbox(
    lat=filtered_combined_df['Latitude'], 
    lon=filtered_combined_df['Longitude'],  
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10, 
        color='blue'
    ),
    name="Stations",                          # Legend name
    hoverinfo='text',
    text=filtered_combined_df['Name'] + "<br>" + filtered_combined_df['Address']  # Tooltip text
))

# Update the layout for the map
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=reference_point['latitude'], lon=reference_point['longitude']),
        zoom=12
    ),
    title="Map of Chosen Stations and Reference Point",
    title_x=0.5,                             # Title establieshment
    height=600
)


fig.show()

#### Build the Model

In [38]:
import heapq

# Weight for distance (Euro)
t = 100

# Calculate the total cost dynamically
filtered_combined_df['Total Cost'] = (Required_energy * filtered_combined_df['Cost_per_Unit']) + (t * filtered_combined_df['Distance'])

# Check required columns and find the top 5 cheapest stations
if 'Total Cost' in filtered_combined_df.columns:
    top_5 = heapq.nsmallest(
        5,
        filtered_combined_df[['Name', 'Address', 'Latitude', 'Longitude', 'Total Cost', 'Distance', 'Cost_per_Unit', 'Available']].itertuples(index=False),
        key=lambda x: x[4]                   # Index 4 corresponds to 'Total Cost'
    )

    # Convert the result to a DataFrame
    cheapest_stations_df = pd.DataFrame(top_5, columns=['Name', 'Address', 'Latitude', 'Longitude', 'Total Cost', 'Distance', 'Cost_per_Unit', 'Available'])

    # Reset the index to start from 1
    cheapest_stations_df.index = range(1, len(cheapest_stations_df) + 1)
    
    
    # Display the results
    print("\nTop 5 Cheapest Stations in order:")

# Display the resulting DataFrame
cheapest_stations_df.head()


Top 5 Cheapest Stations in order:


,Name,Address,Latitude,Longitude,Total Cost,Distance,Cost_per_Unit,Available
1,REVOS,P-1/52a Budh Vihar Phase-1,28.710874,77.084680,1032.0,4.92,2.00,1
2,Singh Service Station,GT Karnal Road DELHI-110075,28.691400,77.189300,1535.5,9.82,2.05,2
3,REVOS,"473, Britannia Rd, Tri Nagar, Delhi, 110034zypp",28.680721,77.155315,2208.0,8.58,5.00,1
4,REVOS,"Kh.No 65, Balimiki Mandir ke samne",28.744759,77.114293,2743.0,0.43,10.00,1
5,REVOS,B-4/38 ground floor sector-17 Rohini,28.744198,77.120257,2790.0,0.90,10.00,1


#### Display 5 Best Stations on Map

In [39]:
# Create a map figure
fig = go.Figure()

# Add the reference point
fig.add_trace(go.Scattermapbox(
    lat=[reference_point['latitude']],  
    lon=[reference_point['longitude']],  
    mode='markers+text',
    marker=dict(size=15, color='red'),
    text=["Reference Point"],
    textposition="top right",
    name="Reference Point"
))

# Add 5 cheapest stations to the map
fig.add_trace(go.Scattermapbox(
    lat=cheapest_stations_df['Latitude'], 
    lon=cheapest_stations_df['Longitude'],  
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=[
        f"Name: {row['Name']}<br>"
        f"Address: {row['Address']}<br>"
        f"Distance: {row['Distance']} km<br>"
        f"Cost per Kwh: {row['Cost_per_Unit']} <br>"
        f"Total Cost: {row['Total Cost']}"
        for _, row in cheapest_stations_df.iterrows()
    ],
    textposition="bottom right",
    name="Cheapest Stations",
))


# Update the map layout
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=cheapest_stations_df['Latitude'].mean(), lon=cheapest_stations_df['Longitude'].mean()),  # Center the map
        zoom=12
    ),
    title="Top 5 Cheapest Stations and Reference Point",
    title_x=0.5,  # Center the title
    height=600
)

# Show the map
fig.show()



#### Charging Time Calculation

In [40]:
# Calculate charging time
Charging_time = Required_energy / charging_speed_vehicle
Charging_time = round(Charging_time, 2)
# Display charging time
print(f'\nCharging time is : {Charging_time} hours')


Charging time is : 6.75 hours


## Step 3: Real world scenarios

#### Thermal Management

In [50]:
Temperature = int(input('\nPlease enter the temperature (numbers only): '))
if  Temperature > 10:
    print(f'\n It is fine there is no need to worry about energy lose because of temperature! You need same energy.')
    Required_energy_season = Required_energy
elif  Temperature <= -20:
    print('\n There is no way the temperature decreased there, you are in the Netherlands!')
elif  0 < Temperature <= 10:
    More_required_energy = (Required_energy * 0.2)
    Required_energy_season = Required_energy + More_required_energy
    print(f'\n Since the temperature is low, you require {More_required_energy} kwh more energy.')
elif  -10 < Temperature <= 0:
    More_required_energy = (Required_energy * 0.4)
    Required_energy_season = Required_energy + More_required_energy
    print(f'\n Since the temperature is very low, you require {More_required_energy} kwh more energy.')
elif  -20 < Temperature <= -10:
    More_required_energy = (Required_energy * 0.5)
    Required_energy_season = Required_energy + More_required_energy 
    print(f'\n Since the temperature is extremely low, you require {More_required_energy} kwh more energy.')


Please enter the temperature (numbers only): 11

 It is fine there is no need to worry about energy lose because of temperature! You need same energy.


#### Standby Power Consumption

In [49]:
Power_cooling = 5                                                # Power for cooing/heating system for one hour
Power_electronics = 2                                            # Power for electronics (Sensors, GPS, etc.) for one hour
Energy_day = (Power_cooling + Power_electronics) * 8             # Energy for a working day (8 hours)

Startup_energy_consumption = 3                                   # Energy for start up engine   

Reenergy_standby_day = energy_day + startup_energy_consumption     # Total amount of standby energy consumption 

print(f"\n The total standby power consumption energy is {energy_standby_day} kWh")


 The total standby power consumption energy is 59 kWh


59